In [1]:
from flask import Flask,jsonify,render_template
from flask_cors import CORS
import json
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import datetime as dt

make flask server and do sql stuff

In [12]:
app = Flask(__name__)
CORS(app)
with open("../restaurant_data.json", 'r') as file:
    data = json.load(file)

restaurnt_df = pd.DataFrame(data)

engine = create_engine("sqlite:///Resources/votes.sqlite")
base = automap_base()
base.prepare(autoload_with=engine)
types = base.classes.Favorite_Cuisines
session = Session(bind=engine)


def pull_vote_totals():
    boop = session.query(types).all()
    democracy = {"mexican":0, "italian": 1, "thai":0}
    print("losser")
    
    for i in boop:
        print(i.restaurant_type)
        democracy[i.restaurant_type] = i.votes_cast   
        print(i.votes_cast) 
    
    return democracy
    

def add_vote_to_db(vote):
    
    session.update({vote: vote+1}) #not how it wokrs but i cant be bothered rn
    tp = session.query(types).filter(types.restaurant_type == vote).one()
    tp.votes_cast += 1
    session.commit()
    print(f"added {vote} to db")
    

@app.route("/")
def home():
    return render_template('index.html')

@app.route("/map")
def dontwait():
    return render_template('map.html')

@app.route("/chart")
def justkiddingyoucanwait():
    return render_template('chart.html')

@app.route("/btc")
def nvm():
    return render_template('btc.html')

@app.route("/vote<vote>")
def noway(vote):
    add_vote_to_db(vote)
    return vote
    
@app.route("/voteTot")
def jose():
    voteTots = pull_vote_totals()
    
    return json.dumps(voteTots)



if(__name__=="__main__"):
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
